<a href="https://colab.research.google.com/github/LiamSwick/Credit-Score-Project/blob/main/Credit_Score_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import kagglehub
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from scipy.stats import zscore
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc, f1_score

In [7]:
# Load in the dataset from Kaggle
path = kagglehub.dataset_download("parisrohan/credit-score-classification")

print("Path to dataset files:", path)

df = pd.read_csv(path + "/train.csv")

100%|██████████| 9.51M/9.51M [00:00<00:00, 169MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/parisrohan/credit-score-classification/versions/1


/tmp/ipython-input-7-3404476064.py:6: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + "/train.csv")


In [8]:
# Set the display options so that we can see all of the columns in the dataset.
pd.set_option('display.max_columns', None)

In [9]:
# Check for NAN values before we begin cleaning
print(df.isna().sum())

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64


In [10]:
# Create a copy of the dataset that we can edit and still preserve the original
#df.info()
df_copy = df.copy()

In [11]:
# Call out purely numerical columns and typecast all the values in these columns to numerical values.
num_cols = ['Age','Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Monthly_Balance']
for col in num_cols:
  df_copy[col] = pd.to_numeric(df_copy[col], errors = 'coerce')
df_copy.isna().sum()

,0
ID,0
Customer_ID,0
Month,0
Name,9985
Age,4939
SSN,0
Occupation,0
Annual_Income,6980
Monthly_Inhand_Salary,15002
Num_Bank_Accounts,0


In [12]:
# Create function to fill NAN values accurately.
def clean_by_CID(dataframe):
    """
    This function will clean the columns that have a non-zero amount of NAN values.
    It groups by CID and does forward and backward filling for NAN values.

    Arguments:
    (pd.DataFrame): The input is the dataframe you want to clean.

    Returns:
    (pd.DataFrame): The cleaned dataframe (clean in the sense that the NAN values have been filled).
    """

    na_mask = dataframe.isna().sum() > 0
    for col in dataframe.columns[na_mask]:
        dataframe[col] = dataframe.groupby('Customer_ID')[col].ffill().bfill()
    return dataframe

clean_by_CID(df_copy)
print(df_copy.isna().sum())

ID                          0
Customer_ID                 0
Month                       0
Name                        0
Age                         0
SSN                         0
Occupation                  0
Annual_Income               0
Monthly_Inhand_Salary       0
Num_Bank_Accounts           0
Num_Credit_Card             0
Interest_Rate               0
Num_of_Loan                 0
Type_of_Loan                0
Delay_from_due_date         0
Num_of_Delayed_Payment      0
Changed_Credit_Limit        0
Num_Credit_Inquiries        0
Credit_Mix                  0
Outstanding_Debt            0
Credit_Utilization_Ratio    0
Credit_History_Age          0
Payment_of_Min_Amount       0
Total_EMI_per_month         0
Amount_invested_monthly     0
Payment_Behaviour           0
Monthly_Balance             0
Credit_Score                0
dtype: int64


In [13]:
# Begin to clean values that are not labeled NAN, starting with static data columns.
def clean_static_categorical(df, id_col, static_cols):
    """
    For each static column, replace values that don't match the group's majority value.

    Args:
        df (pd.DataFrame): The dataset
        id_col (str): The column identifying the group (e.g. 'Customer_ID')
        static_cols (list): List of categorical columns expected to be static within group

    Returns:
        pd.DataFrame: Cleaned dataframe with additional column:
            - {col}_clean: cleaned version
    """
    df_clean = df.copy()

    for col in static_cols:
        # compute mode per group
        mode_series = (
            df_clean.groupby(id_col)[col]
            .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
            .rename(f'{col}_majority')
        )

        # merge majority value back
        df_clean = df_clean.merge(mode_series, on=id_col)

        # replace inconsistent values
        df_clean[f'{col}_clean'] = np.where(
            df_clean[col] != df_clean[f'{col}_majority'],
            df_clean[f'{col}_majority'],
            df_clean[col]
        )

        # drop helper majority column
        df_clean = df_clean.drop(columns=[f'{col}_majority'])

    return df_clean

id_col = 'Customer_ID'
static_cols = ['Name','Age','SSN','Occupation','Annual_Income','Monthly_Inhand_Salary',
                   'Type_of_Loan','Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate',
                   'Num_of_Loan', 'Type_of_Loan', 'Num_Credit_Inquiries','Credit_Mix',
                   'Payment_of_Min_Amount','Total_EMI_per_month', 'Outstanding_Debt']

df_clean = clean_static_categorical(df_copy, id_col, static_cols)
#df_clean.head(25)

In [14]:
# Check for non-NAN erronoues values in the numerical data (i.e. data points that are out of a logical range)
#for col in num_cols:
  #print(df_copy[col].agg(['min','median', 'max']))

In [16]:
# Replace outlier values in each group with the appropriate metric

def clean_outliers(df, id_col, num_cols):
  """
  This function will group by CID and replace outlier values with the appropriate metric.
  This will likely be the mean or median value for each group. To be able to apply a loop
  that consistenly cleans each different column with different ranges, we will use z-score
  comparison.

  Arguments:
  df (pd.DataFrame): The dataset
  id_col (str): The column identifying the group (e.g. 'Customer_ID')
  num_cols (list): List of numerical columns to clean

  Returns:
  pd.DataFrame: Cleaned dataframe

  """
  df_clean = df.copy()
  for cols in num_cols:
    # Compute z-scores within each group
    df_clean[f'{col}_zscore'] = df_clean.groupby(id_col)[col].transform(lambda x: zscore(x, nan_policy='omit'))

    #Flag values where abs(z) > 3)
    df_clean[f'{col}_is_outlier'] = df_clean[f'{col}_zscore'].abs() > 3

  df_clean = df_clean.merge(z_score_series, on=id_col)

  return df_clean

df_copy = clean_outliers(df_copy,id_col, num_cols)
head(df_copy)

TypeError: 'SeriesGroupBy' object is not callable

In [ ]:
data = {'values': [10, 20, 30, 40, 50]}
df = pd.DataFrame(data)

# Calculate z-scores
df['z_score'] = zscore(df['values'])
print(df)

In [ ]:
 for col in num_cols:
  plt.figure(figsize=(5,5))
  sns.boxplot(df_copy[col])
  plt.show()

In [ ]:
# Data integrity step
df_copy = df_copy.drop(['ID', 'Customer_ID','Name','SSN',], axis=1)

df_copy = df_copy.round(2) # Most of this numerical data only makes sense to the hundreths place (cents)

In [ ]:
# Now all we have to do is clean/process the dates and categorical data so we can begin doing data analysis
df_copy.head()

In [ ]:
subset = df_copy.drop(columns = ['Month', 'Occupation', 'Type_of_Loan', 'Credit_Mix', 'Credit_History_Age',
                                 'Payment_of_Min_Amount','Payment_Behaviour','Credit_Score'])
corr = subset.corr()
#plt.figure(figsize=(10,8))
#sns.heatmap(corr, annot = True)
#plt.show()

In [ ]:
test = "22 years and 4 months"
def process_history(string):

  spoopy = string.split(' ')
  spoopy = pd.Series(spoopy)
  nums = [spoopy[0], spoopy[3]]
  nums = pd.to_numeric(nums)
  years = nums[0] + (nums[1]/12)
  years = round(years, 2)
  return years


In [ ]:
for i in range(np.size(df_copy['Credit_History_Age'])):
  history_string = df_copy.loc[i, 'Credit_History_Age']
  df_copy.loc[i, 'Credit_History_Age'] = process_history(history_string)

In [ ]:
df_copy['Occupation'] = df['Occupation'].astype('category')
df_copy['Type_of_Loan'] = df['Type_of_Loan'].astype('category')
df_copy['Credit_Mix'] = df['Credit_Mix'].astype('category')
df_copy['Credit_Score'] = df['Credit_Score'].astype('category')

In [ ]:
df_copy['Occupation'] = df_copy['Occupation'].replace(to_replace = '_______', method = 'ffill')
df_copy['Type_of_Loan'] = df_copy['Type_of_Loan'].replace(to_replace = '_', method = 'ffill')
df_copy['Credit_Mix'] = df_copy['Credit_Mix'].replace(to_replace = '_', method = 'ffill')
df_copy['Payment_of_Min_Amount'] = df_copy['Payment_of_Min_Amount'].replace(to_replace = '_', method = 'ffill')
df_copy['Payment_Behaviour'] = df_copy['Payment_Behaviour'].replace(to_replace = '_', method = 'ffill')

In [ ]:
df_copy.head(15)

In [ ]:
print(df_copy['Occupation'].unique())